In [ ]:
#  Aula Prática – Integração entre PostgreSQL (Pagila) e APIs com Python

# Nesta atividade, você irá:
# - Consultar dados da base Pagila com `psycopg2`
# - Integrar dados climáticos e populacionais usando APIs externas
# - Analisar, transformar e visualizar os dados
# - Praticar o uso de Pandas, APIs, SQL e operações avançadas como `lambda`, `groupby`, `merge`, entre outras

In [ ]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import requests
import seaborn as sns
from dotenv import load_dotenv

from tabulate import tabulate

In [ ]:
# Carregar variáveis de ambiente
load_dotenv()

# Obter as variáveis de ambiente
db = os.getenv("PG_DB")
user = os.getenv("PG_USER")
password = os.getenv("PG_PASSWORD")
host = os.getenv("PG_HOST")
port = os.getenv("PG_PORT")
sslmode = os.getenv("PG_SSLMODE")

conn = psycopg2.connect(
    dbname=db, user=user, password=password, host=host, port=port, sslmode=sslmode
)

query_sql = "SELECT VERSION()"

cur = conn.cursor()
cur.execute(query_sql)

version = cur.fetchone()[0]
print(f"PostgreSQL version: {version}")

# # Fechar a conexão
# cur.close()
# conn.close()

PostgreSQL version: PostgreSQL 16.8 on x86_64-pc-linux-gnu, compiled by gcc (GCC) 14.2.1 20240912 (Red Hat 14.2.1-3), 64-bit


In [8]:
def run_query(sql, conn=conn):
    """
    Executes a SQL query on the provided database connection and returns the result as a DataFrame.

    Args:
        sql (str): The SQL query to be executed.
        conn (psycopg2.extensions.connection): The database connection object.

    Returns:
        pandas.DataFrame: A DataFrame containing the results of the SQL query.
    """
    return pd.read_sql_query(sql, conn)

In [9]:
def buscar_clima(cidade):
    api_key = os.getenv("WEATHER_KEY")
    try:
        url = "http://api.weatherapi.com/v1/current.json?" f"key={api_key}&q={cidade}"
        resposta = requests.get(url, timeout=10)
        return resposta.json()["current"]["temp_c"]
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return None


cidade = buscar_clima("São Paulo")

print(f"A temperatura atual em São Paulo é {cidade}°C")

A temperatura atual em São Paulo é 16.7°C


In [ ]:
def merge_client_weather(df_cidades):
    # Aplicar em uma amostra das cidades
    amostra = df_cidades["city"].drop_duplicates().head(10)
    df_clima = pd.DataFrame({"cidade": amostra})
    df_clima["temperatura"] = df_clima["cidade"].apply(buscar_clima)
    # print(df_clima.head())

    # Exemplo de merge
    df_analise = df_cidades.merge(
        df_clima, how="left", left_on="city", right_on="cidade"
    )
    print(tabulate(df_analise.head(10), headers="keys", tablefmt="pretty"))
    return df_analise

In [11]:
## https://github.com/devrimgunduz/pagila/tree/master


#   Exercício 1 – Temperatura Média das Capitais dos Clientes
# 	•	Recupere as cidades dos clientes com mais de 10 transações.
# 	•	Use a WeatherAPI para buscar a temperatura atual dessas cidades.
# 	•	Calcule a temperatura média ponderada por número de clientes.
# 	•	Insight esperado: quais cidades concentram clientes e temperaturas extremas?

# ⸻


# Exemplo de query
query_cidades = """
SELECT 
    ci.city, 
    COUNT(p.payment_id) AS num_transacoes
FROM payment p
    JOIN customer c ON p.customer_id = c.customer_id
    JOIN address a ON c.address_id = a.address_id
    JOIN city ci ON a.city_id = ci.city_id
GROUP BY ci.city
HAVING COUNT(p.payment_id) > 10
ORDER BY num_transacoes DESC;
"""

df_cidades = run_query(query_cidades)

df_analise = merge_client_weather(df_cidades)

C:\Users\dgeis\AppData\Local\Temp\ipykernel_1468\968903793.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(sql, conn)


Error fetching weather data: 'current'
+---+------------------+----------------+------------------+-------------+
|   |       city       | num_transacoes |      cidade      | temperatura |
+---+------------------+----------------+------------------+-------------+
| 0 |     Springs      |       58       |     Springs      |     9.4     |
| 1 |      Aurora      |       50       |      Aurora      |    27.7     |
| 2 |      London      |       48       |      London      |    13.2     |
| 3 |   Saint-Denis    |       46       |   Saint-Denis    |    20.0     |
| 4 |    Cape Coral    |       45       |    Cape Coral    |    25.6     |
| 5 |    Molodetno     |       42       |    Molodetno     |     nan     |
| 6 |      Tanza       |       42       |      Tanza       |    28.9     |
| 7 |     Changhwa     |       41       |     Changhwa     |    20.8     |
| 8 |    Changzhou     |       40       |    Changzhou     |    22.9     |
| 9 | Ourense (Orense) |       40       | Ourense (Orense) | 

In [ ]:
#   Exercício 2 – Receita Bruta em Cidades com Clima Ameno
# 	•	Calcule a receita bruta por cidade.
# 	•	Use a WeatherAPI para consultar a temperatura atual.
# 	•	Filtre apenas cidades com temperatura entre 18°C e 24°C.
# 	•	Resultado: qual o faturamento total vindo dessas cidades?


# ⸻

In [ ]:
#   Exercício 3 – Aluguel de Filmes por Região e População
# 	•	Identifique os países dos clientes com maior número de aluguéis.
# 	•	Use a REST Countries API para obter a população desses países.
# 	•	Calcule o número de aluguéis por 1.000 habitantes.
# 	•	Análise: quais países são mais “cinéfilos” proporcionalmente?

# ⸻

#   Exercício 4 – Filmes Mais Populares em Cidades Poluídas
# 	•	Liste as 10 cidades com maior número de clientes.
# 	•	Use a AirVisual API para consultar o AQI dessas cidades.
# 	•	Relacione os filmes mais alugados em cidades com AQI > 150.
# 	•	Discussão: poluição impacta preferências de filmes?

# ⸻

#   Exercício 5 – Clientes em Áreas Críticas
# 	•	Recupere os clientes com endereço em cidades com AQI acima de 130.
# 	•	Combine nome do cliente, cidade, país, temperatura e AQI.
# 	•	Classifique os clientes em “zona de atenção” com base nos critérios ambientais.

# ⸻

#   Exercício 6 – Receita por Continente
# 	•	Use a REST Countries API para mapear o continente de cada país.
# 	•	Agrupe a receita total por continente.
# 	•	Exiba os resultados em um gráfico de pizza com matplotlib.

# ⸻

#   Exercício 7 – Tempo Médio de Aluguel vs Clima
# 	•	Calcule o tempo médio de aluguel por cidade (entre rental_date e return_date).
# 	•	Combine com a temperatura atual dessas cidades.
# 	•	Visualize a correlação entre temperatura e tempo médio de aluguel (scatterplot + linha de tendência).

# ⸻

#   Exercício 8 – Perfil de Clima por Cliente
# 	•	Para cada cliente, crie um perfil com:
# 	•	cidade, temperatura, AQI, total de aluguéis, gasto total.
# 	•	Agrupe os perfis por faixa etária (simulada ou fictícia) e avalie padrões.
# 	•	Objetivo: conectar comportamento de consumo e ambiente.

# ⸻

#   Exercício 9 – Exportação Inteligente
# 	•	Gere um relatório Excel com os seguintes critérios:
# 	•	Clientes de países com temperatura < 15°C
# 	•	AQI acima de 100
# 	•	Receita individual > média geral
# 	•	Utilize OpenPyXL e organize em múltiplas abas: Clientes, Temperaturas, Alertas.

# ⸻

#   Exercício 10 – API Cache Inteligente (Desafio)
# 	•	Implemente uma lógica que salve os dados de clima e AQI localmente em CSV.
# 	•	Ao consultar novamente a mesma cidade, busque do CSV ao invés da API.
# 	•	Evite chamadas redundantes — bom para práticas de performance e economia de requisições.